In [2]:
import pandas as pd
import numpy as np
import preprocessor as p
import requests

from tqdm.notebook import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', None)

In [3]:
raw = pd.read_csv('data/data.csv')
raw.shape

(108525, 6)

In [4]:
raw.head()

,tweet_id,created_at,author_id,username,text,reference_type
0,1620209968682254336,2023-01-30T23:58:53.000Z,185508421,rachmanfahadi,RT @NephiLaxmus: Saya\n\n- Pabrik motor listrik ini punya?\n+ Saya\n- Ojek online ini?\n+ Saya\n- Pembangkit listriknya?\n+ Saya\n- Pemasok batu ba…,['retweeted']
1,1620209586115604480,2023-01-30T23:57:22.000Z,458497906,66fathur,RT @NephiLaxmus: Saya\n\n- Pabrik motor listrik ini punya?\n+ Saya\n- Ojek online ini?\n+ Saya\n- Pembangkit listriknya?\n+ Saya\n- Pemasok batu ba…,['retweeted']
2,1620209295337078786,2023-01-30T23:56:12.000Z,954981728350695425,erusdhiana0118,"RT @giginpraginanto: Dia yang memproduksi, dia juga yang kasih subsidi. Dia pasti makin kaya raya dan berkuasa kalau program 3 periode berj…",['retweeted']
3,1620209006261460993,2023-01-30T23:55:03.000Z,1148010397951094784,AloelAsheen,"RT @giginpraginanto: Dia yang memproduksi, dia juga yang kasih subsidi. Dia pasti makin kaya raya dan berkuasa kalau program 3 periode berj…",['retweeted']
4,1620208586558435330,2023-01-30T23:53:23.000Z,1379546070783430661,DM_N4BYL,"RT @Nailazaara__: Erick Thohir melihat perkembangan renovasi ruang klinik dlm pemenuhan pelayanan atas pentingnya kesehatan pegawai.\n""Ke de…",['retweeted']


In [5]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108525 entries, 0 to 108524
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tweet_id        108525 non-null  int64 
 1   created_at      108525 non-null  object
 2   author_id       108525 non-null  int64 
 3   username        108525 non-null  object
 4   text            108525 non-null  object
 5   reference_type  83335 non-null   object
dtypes: int64(2), object(4)
memory usage: 5.0+ MB


# 1. Data Preprocessing
---

## 1.1 Filter Tweets

Drop retweeted Tweets to diminish duplication

In [6]:
raw['reference_type'].value_counts()

['retweeted']               66528
['replied_to']              14489
['quoted']                   2230
['quoted', 'replied_to']       88
Name: reference_type, dtype: int64

We will drop 66528 retweeted Tweets

In [15]:
df = raw[raw['reference_type'] != "['retweeted']"].reset_index(drop=True)

In [16]:
df.shape

(41997, 6)

Drop duplicated tweets and keep the first occurance

In [18]:
duplicate = df[df.duplicated('text')]
print('Total duplicated tweets:',duplicate.shape[0])

Total duplicated tweets: 2348


In [19]:
df.drop_duplicates(subset=['text'], inplace=True)

In [20]:
df.shape

(39649, 6)

## 1.2 Text Preprocessing

In [21]:
# Normalisasi Kata
url1 = (r'https://raw.githubusercontent.com/ksnugroho/klasifikasi-spam-sms/master/data/key_norm.csv')
url2 = (r'https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv')
url3 = (r'https://raw.githubusercontent.com/okkyibrohim/id-abusive-language-detection/master/kamusalay.csv')
url4 = (r'https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_slang_words.txt')
r = requests.get(url4)

kamus_normalisasi = pd.read_csv(url1)
kamus_alay1 = pd.read_csv(url2)
kamus_alay2 = pd.read_csv(url3, names=['slang','formal'])
kamus_slang = pd.json_normalize(r.json()).transpose().reset_index()
kamus_slang.columns = ['slang','formal']

def normalisasi(text):
  text = ' '.join([kamus_normalisasi[kamus_normalisasi['singkat'] == word]['hasil'].values[0] if (kamus_normalisasi['singkat'] == word).any() else word for word in text.split()])
  text = ' '.join([kamus_alay1[kamus_alay1['slang'] == word]['formal'].values[0] if (kamus_alay1['slang'] == word).any() else word for word in text.split()])
  text = ' '.join([kamus_alay2[kamus_alay2['slang'] == word]['formal'].values[0] if (kamus_alay2['slang'] == word).any() else word for word in text.split()])
  text = ' '.join([kamus_slang[kamus_slang['slang'] == word]['formal'].values[0] if (kamus_slang['slang'] == word).any() else word for word in text.split()])
  return text

def normalisasi_slang(text):
  text = ' '.join([kamus_alay1[kamus_alay1['slang'] == word]['formal'].values[0] if (kamus_alay1['slang'] == word).any() else word for word in text.split()])
  text = ' '.join([kamus_alay2[kamus_alay2['slang'] == word]['formal'].values[0] if (kamus_alay2['slang'] == word).any() else word for word in text.split()])
  text = ' '.join([kamus_slang[kamus_slang['slang'] == word]['formal'].values[0] if (kamus_slang['slang'] == word).any() else word for word in text.split()])
  return text

In [22]:
# Pipeline
def preprocessing(text):
  text = text.replace('\n', ' ').replace('\r', '') # replace new line into space
  text = text.replace('&amp;', ' ')
  p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.HASHTAG, p.OPT.RESERVED)
  text = p.clean(text)
  text = normalisasi(text)
  #text = normalisasi_slang(text)
  return text

In [23]:
# Start preprocessing
df['text_cleaned'] = df['text'].progress_apply(preprocessing)

  0%|          | 0/39649 [00:00<?, ?it/s]

In [24]:
df[['text','text_cleaned']].sample(10)

,text,text_cleaned
29808,"@Eno_Bening elon kesalip karna harga saham tesla turun kali, baca lagi ah no","elon kesalip karena harga saham tesla turun kali, baca lagi ah nomor"
688,"#MostPopuler Daftar Harga Motor Listrik dengan Subsidi Rp 7 Juta, Ada yang Rp 6 Jutaan! https://t.co/m0qmZ52reT","Daftar Harga Motor Listrik dengan Subsidi Rp 7 Juta, Ada yang Rp 6 Jutaan!"
31741,PUPR siapkan SPKLU di 14 rest area tol bagi pemudik mobil listrik https://t.co/4leywnXsFg,PUPR siapkan SPKLU di 14 rest area tol bagi pemudik mobil listrik
27920,"Apakah rezim ini sdh menjadikan rakyat ekonomi lemah sbg ""musuh yg harus dilenyapkan dimuka bumi""?\nSubsidi mobil+motor listrik bukti rezim melindungi kepentingan si kaya, cabut semua subsidi bukti upaya pemusnahan rakyat jelata https://t.co/Wx6Ka2RL1m","Apakah rezim ini sudah menjadikan rakyat ekonomi lemah sebagai ""musuh yang harus dilenyapkan dimuka bumi""? Subsidi mobil+motor listrik bukti rezim melindungi kepentingan sih kaya, pergi semua subsidi bukti upaya pemusnahan rakyat jelata"
7122,"Untuk para capres 2024 ayo adu ide dan gagasan soal hak dasar masyarakat seperti air bersih, transportasi publik, dan layanan administrasi kependudukan yg excellent.\n\nGak usah dakik dakik kosong kaya mobil listrik padahal cuma nolong oligarki nikel https://t.co/tZf9nJF3qr","Untuk para capres 2024 ayo adu ide dan gagasan soal hak dasar masyarakat seperti air bersih, transportasi publik, dan layanan administrasi kependudukan yang excellent. Gak perlu dakik dakik kosong kayak mobil listrik padahal cuma menolong oligarki nikel"
41304,"@cimotyy aahh pngn bngt pnya mobil listrik ..yng pasti jdi hemat bensyinn kan yaak ,cakep pulaaa model nyaa #PakaiMolisYuk 😍","ah ingin banget punya mobil listrik ..yng pasti jadi hemat bensyinn kan iya ,cakep pulaaa model nya 😍"
9855,"Halo Electrizen!\nPada gelaran Indonesia International Motor Show (IIMS) 2023 kali ini, PLN menghadirkan 12 motor listrik hasil konversi dari motor BBM loh. https://t.co/gVQ3zZ5egN","Halo Electrizen! Pada gelaran Indonesia International Motor Show (IIMS) 2023 kali ini, PLN menghadirkan 12 motor listrik hasil konversi dari motor BBM loh."
36819,@Tak_Ngerti_apa2 @Mdy_Asmara1701 🤣🤣 @aniesbaswedan loh yg bilang mobil listrik mengandung emisi lebih besar daripada bus berpenumpang.. 🤣,🤣🤣 loh yang bilang mobil listrik mengandung emisi lebih besar daripada bus berpenumpang.. 🤣
34255,"Electrizen, PLN memastikan kesiapan SPKLU untuk memudahkan penguna kendaraan listrik selama masa mudik lebaran Hari Raya Idul Fitri 1444 Hijriah. #PLN #SPKLUPLN #PakaiMolis #PLN #PowerBeyondGenerations #AyoPakaiEV #SPKLUPLNRamadan #PLNSiagaIdulFitri #PLNEMobilizeYou https://t.co/Cms84r1aIz","Electrizen, PLN memastikan kesiapan SPKLU untuk memudahkan pengguna kendaraan listrik selama masa mudik lebaran Hari Raya Idul Fitri 1444 Hijriah."
11091,"Pengguna kendaraan listrik biasanya ngecas daya di rumah, nah SPKLU tuh biasanya khusus untuk perjalanan jauh. Jadi kayak pom bensin gitu, SPKLU PLN disiapkan di lokasi-lokasi yang memang menjadi jalur perjalanan jauh, kayak tol trans jawa atau trans sumatera\n#PLNEMobilizeYou https://t.co/VjToBEq9CI","Pengguna kendaraan listrik biasanya ngecas daya di rumah, nah SPKLU tuh biasanya khusus untuk perjalanan jauh. Jadi seperti pom bensin gitu, SPKLU PLN disiapkan di lokasi-lokasi yang memang menjadi jalur perjalanan jauh, seperti tol trans jawa atau trans sumatera"


Remove the duplicated cleaned tweets again

In [29]:
duplicate2 = df[df.duplicated('text_cleaned')]
print('Total duplicated tweets:',duplicate2.shape[0])

Total duplicated tweets: 8538


In [31]:
df.drop_duplicates(subset=['text_cleaned'], inplace=True)

In [32]:
df.shape

(31111, 7)

In [33]:
# Save the prepocessing result
df.to_csv('data/data_preprocessed.csv', index=False)